# Example of how to see your learning parameters in TensorBoard

The point of this code example is to show you how to record statistics during learning and display them using TensorBoard

Refer to the following for additional functions (can record data/images, graphs, etc.)
  * https://tensorboardx.readthedocs.io/en/latest/tensorboard.html

In [ ]:
import torch
from torch.autograd import Variable
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

#######################################################################################

# our neural net class
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc1 = nn.Linear(2,1,True)
    def forward(self, x):
        x = self.fc1(x)
        return x
    
# make the neuron
net = Perceptron()
# net.cuda() # GPU acceleration

#######################################################################################

# make some synthetic data
N = 100
Nhalf = int( N / 2 )
data = torch.randn(N,2) 
for i in range(Nhalf): # class 1
    data[i,0] = data[i,0] * 0.1 + 0.2
    data[i,1] = data[i,1] * 0.1 + 0.2
for i in range(Nhalf): # class 2
    data[i+Nhalf,0] = data[i+Nhalf,0] * 0.1 + -0.2
    data[i+Nhalf,1] = data[i+Nhalf,1] * 0.1 + 0.8
    
# labels
L = torch.ones(N)
for i in range(Nhalf): # class 1
    L[i] = 1
for i in range(Nhalf): # class 2
    L[i+Nhalf] = -1
    
#######################################################################################

from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard
# %reload_ext tensorboard
writer = SummaryWriter()

#######################################################################################

# criteria fx
def criterion(out,label):
    return (label - out)**2

# setup SGD optimization
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)

# train
for epoch in range(500):
    accum=0 # sum up error per epoch
    for i in range(N):
        X = Variable(data[i,:])
        Y = Variable(L[i])
        optimizer.zero_grad()
        outputs = net(X)
        loss = criterion(outputs, Y)
        accum=accum+loss # accumulate that error
        loss.backward()
        optimizer.step()
    accum=accum/N # divide by the number of objects we evaluated over
    writer.add_scalar('loss', accum, epoch) # post that to our file for TensorBoard to analyze
        
#######################################################################################

%tensorboard --logdir runs        